In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
import cv2




In [2]:
intial_data = pd.read_csv('train.csv')
intial_data.head()

,im_name,label
0,00016cd.jpg,6
1,0001808.jpg,2
2,0002399.jpg,1
3,0003973.jpg,3
4,00061cc.jpg,4


In [13]:
from skimage import feature, io, color

def compute_hog_features(image):
    image_read= io.imread(image)
    gray_image = color.rgb2gray(image_read)
    # Adjust parameters like pixels_per_cell and cells_per_block based on your specific needs
    fd = feature.hog(gray_image, visualize=False, block_norm='L2-Hys', pixels_per_cell=(16, 16), cells_per_block=(1, 1))
    return fd

In [14]:
import os 
image_array=[]
path= '/Users/Admin/Documents/HKU Fintech Work/Year 2/Year 2 Sem 2/COMP3314 Machine Learning/Group Project/COMP3314-Group-Project/train_ims' #change path as required to get to train_ims
os.chdir(path)

for jpg in intial_data['im_name']:
    image_format= compute_hog_features(jpg)
    image_array.append(image_format)

In [15]:
print(len(image_array[1]))

36


In [16]:
sorted_intial_data= intial_data.sort_values(by=['im_name'])


In [17]:
sorted_intial_data['image_info']=image_array
sorted_intial_data=sorted_intial_data[['im_name','image_info','label']]
sorted_intial_data.head()

,im_name,image_info,label
0,00016cd.jpg,"[0.35740033007451505, 0.35740033007451505, 0.3...",6
1,0001808.jpg,"[0.33829065745884185, 0.33829065745884185, 0.3...",2
2,0002399.jpg,"[0.3687752209318712, 0.3687752209318712, 0.368...",1
3,0003973.jpg,"[0.3578438528772558, 0.3578438528772558, 0.285...",3
4,00061cc.jpg,"[0.3469809003367331, 0.3469809003367331, 0.346...",4


In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X= np.array(sorted_intial_data['image_info'].tolist()) 
Y= np.array(sorted_intial_data['label'].tolist())  

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)


In [19]:
#Train SVM with necessary Grids:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'kernel': ['rbf', 'sigmoid'], 'gamma': [0.01, 0.001, 0.0001]}

grid_search= GridSearchCV(SVC(), param_grid, refit=True, verbose=3, cv=5)
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits
[CV 1/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.343 total time=  46.8s
[CV 2/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.348 total time=  46.9s
[CV 3/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.342 total time=  46.3s
[CV 4/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.346 total time=  46.7s
[CV 5/5] END .....C=0.1, gamma=0.01, kernel=rbf;, score=0.343 total time=  46.1s
[CV 1/5] END .C=0.1, gamma=0.01, kernel=sigmoid;, score=0.313 total time=  40.0s
[CV 2/5] END .C=0.1, gamma=0.01, kernel=sigmoid;, score=0.317 total time=  40.0s
[CV 3/5] END .C=0.1, gamma=0.01, kernel=sigmoid;, score=0.308 total time=  39.6s
[CV 4/5] END .C=0.1, gamma=0.01, kernel=sigmoid;, score=0.320 total time=  39.7s
[CV 5/5] END .C=0.1, gamma=0.01, kernel=sigmoid;, score=0.313 total time=  39.6s
[CV 1/5] END ....C=0.1, gamma=0.001, kernel=rbf;, score=0.304 total time=  53.1s
[CV 2/5] END ....C=0.1, gamma=0.001, kernel=rbf;

GridSearchCV(cv=5, estimator=SVC(),
             param_grid={'C': [0.1, 1, 10], 'gamma': [0.01, 0.001, 0.0001],
                         'kernel': ['rbf', 'sigmoid']},
             verbose=3)

In [20]:
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print(f"Best parameters: {best_parameters}")
print(f"Best cross-validation score: {best_score}")

Best parameters: {'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}
Best cross-validation score: 0.37140000000000006
